In [7]:
import get_started_doing_something_variables
import get_stopped_doing_something_variables
import get_all_vars

import pandas as pd
import read_a_combination_of_variables
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
from helpers.s3_bucket_utils import S3BucketUtils
from helpers import settings
import ast
import yaml
import numpy as np

bucket = S3BucketUtils()

with open(r'./parameters/for_properly_used_inquiries_vars.yaml') as file:
    for_properly_used_inquiries_vars = yaml.load(file, Loader=yaml.FullLoader)

def get_key_based_on_value_in_a_dict(dict_, value_of_interest):
    for key, value in dict_.items():
        if value == value_of_interest:
            return key

def get_model_number(var_, model_names):
    if len(model_names[model_names['model_name']==var_]['model_number'])>0:
        model_number = model_names[model_names['model_name']==var_]['model_number'].iloc[0]
    else:
        multiple_vars = model_names[(model_names['model_name'].apply(lambda x: ',' in x))]
        for i in range(0, len(multiple_vars)):
            if var_ in multiple_vars['model_name'].iloc[i]:
                model_number = multiple_vars['model_number'].iloc[i]
                break
    return model_number

def get_pairs_of_variables(churn_based_on_behaviour_dir, date_dir, model_type):
    vars_periods_to_look_at_thresholds_to_use = \
    bucket.load_csv_from_s3(file_name = churn_based_on_behaviour_dir + 'data/' + date_dir +\
                     '/vars_periods_to_look_at_thresholds_to_use.csv')

    period_to_look_at =\
    vars_periods_to_look_at_thresholds_to_use[['variable_base_name', 'period_to_look_at', 'number_of_months']].\
    apply(lambda x: (x['variable_base_name'], x['period_to_look_at']), axis=1).tolist()


    (all_vars, all_vars_base_names) = get_all_vars.main()
    first_var = dict.fromkeys(all_vars_base_names.values())
    second_var = dict.fromkeys(all_vars_base_names.values())

    if model_type == 'started_doing_something':
        for i in range(0, len(period_to_look_at)):
            first_var[period_to_look_at[i][0]] = 'had_'+period_to_look_at[i][0]+'_'+period_to_look_at[i][1]
            second_var[period_to_look_at[i][0]] = \
            'had_'+period_to_look_at[i][0]+'_before_and_didnt_'+period_to_look_at[i][1]

        for key in first_var.keys():
            if 'changed' in key.lower() and 'inquiry' in key.lower():
                period_to_look_at_tmp = vars_periods_to_look_at_thresholds_to_use[vars_periods_to_look_at_thresholds_to_use['variable_base_name']==\
                                                                                 key]['period_to_look_at'].iloc[0]

                first_var[key] = 'had_'+for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][key]+'_'+period_to_look_at_tmp
        for key in second_var.keys():
            if 'changed' in key.lower() and 'inquiry' in key.lower():
                period_to_look_at_tmp = vars_periods_to_look_at_thresholds_to_use[vars_periods_to_look_at_thresholds_to_use['variable_base_name']==\
                                                                                 key]['period_to_look_at'].iloc[0]

                second_var[key] = 'had_'+for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][key]+'_before_and_didnt_'+period_to_look_at_tmp

    elif model_type == 'stopped_doing_something':
        for i in range(0, len(period_to_look_at)):
            first_var[period_to_look_at[i][0]] = 'had_'+period_to_look_at[i][0]+'_before'
            second_var[period_to_look_at[i][0]] = 'had_'+period_to_look_at[i][0]+'_'+period_to_look_at[i][1]

        for key in first_var.keys():
            if 'changed' in key.lower() and 'inquiry' in key.lower():
                period_to_look_at_tmp = vars_periods_to_look_at_thresholds_to_use[vars_periods_to_look_at_thresholds_to_use['variable_base_name']==\
                                                                                 key]['period_to_look_at'].iloc[0]

                first_var[key] = 'had_'+for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][key]+'_before'
        for key in second_var.keys():
            if 'changed' in key.lower() and 'inquiry' in key.lower():
                period_to_look_at_tmp = vars_periods_to_look_at_thresholds_to_use[vars_periods_to_look_at_thresholds_to_use['variable_base_name']==\
                                                                                 key]['period_to_look_at'].iloc[0]

                second_var[key] = 'had_'+for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][key]+'_'+period_to_look_at_tmp


    return (first_var, second_var)

date_of_analysis = input('date of analysis: ')
date_dir = date_of_analysis.replace('-', '_')

model_type = input('Model type: ')

var_ = input("Variable we're interested in: ")
# had_private_parties_submissions_last_2_months

without_properly_used_vars = input('Without properly used inquiries variables (Yes/No)? ')
if without_properly_used_vars == 'Yes':
    without_properly_used_vars = True
    combinations_of_vars_dir = 'combinations_of_variables_that_are_not_dependent/combinations_and_names_without_properly_used_inquiries_vars'
elif without_properly_used_vars == 'No':
    without_properly_used_vars = False
    combinations_of_vars_dir = 'combinations_of_variables_that_are_not_dependent'

(first_var, second_var) = \
get_pairs_of_variables(churn_based_on_behaviour_dir='churn_analysis_based_on_behaviour/', date_dir=date_dir,\
                       model_type=model_type)
    
if var_ in first_var.values():
    base_var_ = get_key_based_on_value_in_a_dict(dict_=first_var, value_of_interest=var_)
elif var_ in second_var.values():
    base_var_ = get_key_based_on_value_in_a_dict(dict_=second_var, value_of_interest=var_)
else:
    base_var_ = var_
    
if 'properly' in var_:
    base_var_ = for_properly_used_inquiries_vars['changed_inquiry_status_to_properly_used'][base_var_]

date of analysis: 2021-12-01
Model type: started_doing_something
Variable we're interested in: had_catering_submissions_last_3_months
Without properly used inquiries variables (Yes/No)? Yes


In [8]:
model_names = \
bucket.load_csv_from_s3(file_name='churn_analysis_based_on_behaviour/'+combinations_of_vars_dir+'/model_names.csv')
model_number = get_model_number(var_=base_var_, model_names=model_names)

In [9]:
for condition in os.listdir('data/'+date_dir+'/exports/coefficients_and_pvalues/'+model_type+'/'+var_+'/'):
    coefs_and_pvalues_export = pd.DataFrame(columns = ['model_number', 'model_name', 'data_set', 'variable', 'coef', 'exp(coef)', 'p'])
    if without_properly_used_vars == True:
        files = [x for x in os.listdir('data/'+date_dir+'/exports/coefficients_and_pvalues/'+model_type+'/'+var_+'/'+condition+'/') if 'without_properly_used' in x]
    else:
        files = [x for x in os.listdir('data/'+date_dir+'/exports/coefficients_and_pvalues/'+model_type+'/'+var_+'/'+condition+'/') if 'with_properly_used' in x]
    for filename in files:
#         print(filename)
        coefs_and_pvalues = \
        bucket.load_csv_from_s3(file_name='churn_analysis_based_on_behaviour/data/'+date_dir+'/exports/coefficients_and_pvalues/'+model_type+'/'+var_+'/'+\
                               condition+'/'+filename)
        if without_properly_used_vars == True:
            dataset = filename.split('without_properly_used_vars_')[1].split('.csv')[0]
        else:
            dataset = filename.split('with_properly_used_vars_')[1].split('.csv')[0]
            
        idx = len(coefs_and_pvalues_export)
        coefs_and_pvalues_export.loc[idx, 'model_number'] = model_number
        coefs_and_pvalues_export.loc[idx, 'model_name'] = model_names[model_names['model_number']==model_number]['model_name'].iloc[0]
        coefs_and_pvalues_export.loc[idx, 'data_set'] = dataset
        coefs_and_pvalues_export.loc[idx, 'variable'] = var_
#         print(condition)
        if var_ in coefs_and_pvalues['covariate'].unique():
#             print(filename)
            coefs_and_pvalues_export.loc[idx, 'coef'] = \
            coefs_and_pvalues[coefs_and_pvalues['covariate']==var_]['coef'].iloc[0]
            coefs_and_pvalues_export.loc[idx, 'exp(coef)'] = \
            coefs_and_pvalues[coefs_and_pvalues['covariate']==var_]['exp(coef)'].iloc[0]
            coefs_and_pvalues_export.loc[idx, 'p'] = \
            coefs_and_pvalues[coefs_and_pvalues['covariate']==var_]['p'].iloc[0]
        else:
            coefs_and_pvalues_export.loc[idx, 'coef'] = np.nan
            coefs_and_pvalues_export.loc[idx, 'exp(coef)'] = np.nan
            coefs_and_pvalues_export.loc[idx, 'p'] = np.nan
        
    if without_properly_used_vars == True:
        if not os.path.exists('data/'+date_dir+'/exports/'+model_type+'/'+var_+'/without_properly_used_inquiries_vars/'):
            os.makedirs('data/'+date_dir+'/exports/'+model_type+'/'+var_+'/without_properly_used_inquiries_vars/')
        coefs_and_pvalues_export.\
        to_csv('data/'+date_dir+'/exports/'+model_type+'/'+var_+'/without_properly_used_inquiries_vars/'+condition+'_coefs_and_pvalues.csv', index = False)
    else:
        if not os.path.exists('data/'+date_dir+'/exports/'+model_type+'/'+var_+'/with_properly_used_inquiries_vars/'):
            os.makedirs('data/'+date_dir+'/exports/'+model_type+'/'+var_+'/with_properly_used_inquiries_vars/')
        coefs_and_pvalues_export.\
        to_csv('data/'+date_dir+'/exports/'+model_type+'/'+var_+'/with_properly_used_inquiries_vars/'+condition+'_coefs_and_pvalues.csv', index = False)       